In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import scipy.stats as stats

In [18]:
#WNBA Games Started Stats

teams = ["atl", "chi", "conn", "dal", "ind", "lv", "la", "min", "ny", "phx", "sea", "wsh"]

names_start = []
all_data = []
gs = []

for i in teams:
    URL = "https://www.espn.com/wnba/team/stats/_/name/"
    URL = URL + i
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find_all("tr", class_ = "Table__TR Table__TR--sm Table__even")

    i = 0
    j = 0

    for line in results:
        i += 1

    for line in results:
        j += 1
        player_name = line.find('a')
        names_start.append(player_name.text)
        if (j == (i/4 - 1)):
            break

    results = soup.find_all("span")

    j = 0

    for line in results:
        if ("0" in line.text or "1" in line.text or "2" in line.text or "3" in line.text or "4" in line.text or "5" in line.text or "6" in line.text or "7" in line.text or "8" in line.text or "9" in line.text):
            j += 1
            all_data.append(float(line.text))
        if (j == ((i/4 - 1) * 13)):
            break

    for k in range(int(i/4 - 1)):
        gs.append(all_data[k*13 + 1])

In [19]:
#WNBA Salaries
URL = "https://www.spotrac.com/wnba/rankings/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

names_salary = []
raw = []
salaries = []

results = soup.find_all("td", class_ = "rank-name player noborderright")

for line in results:
    player_name = line.find('a', class_ = "team-name")
    names_salary.append(player_name.text)

results = soup.find_all("tr")
for line in results:
    salary = line.find('span', class_ = "info")
    raw.append(salary)

for i in range(1, len(raw)):
    salary = (raw[i].text).replace("  ", "")
    salary = salary.replace("$", "")
    salary = int(salary.replace(",", ""))
    salaries.append(salary)


In [20]:
salaries_sorted = []
games_started = []
data = np.genfromtxt('wnba-player-stats.csv', dtype = (str), skip_header = 1, delimiter = ',')
players = data[:, 0]

i = -1
for player in players:
    if player in names_salary:
        index = names_salary.index(player)
        salaries_sorted.append(salaries[index])
    else:
        salaries_sorted.append(0)
        
    if player in names_start:
        index = names_start.index(player)
        games_started.append(gs[index])
    else:
        games_started.append(0)

[108000, 227900, 72141, 196267, 200000, 228094, 227900, 228094, 196267, 0, 205000, 200000, 227900, 170000, 120000, 72141, 200000, 196267, 185400, 72141, 72141, 180250, 150000, 72141, 170000, 196267, 76297, 150350, 135000, 145000, 142800, 195000, 148400, 190000, 215000, 195000, 159000, 73542, 113300, 72141, 164800, 0, 0, 68552, 130000, 0, 196267, 140000, 72141, 228094, 0, 67042, 155000, 185000, 107040, 170000, 141000, 0, 160000, 90000, 0, 0, 221450, 76297, 140000, 0, 0, 0, 72141, 71441, 196267, 90000, 129150, 160000, 196100, 0, 0, 73211, 0, 0, 67042, 75000, 98000, 115000, 70127, 180200, 76297, 0, 0, 73211, 0, 0, 190550, 0, 70127, 110000, 67042, 0, 0, 0, 71441, 74000, 0, 0, 0, 80000, 0, 73211, 0, 160000, 0, 0, 0, 72141, 0, 0, 0, 0, 116390, 0, 0, 0, 0, 227900, 71441, 0, 0, 0, 0, 0, 0, 0, 196000, 115000, 0, 80000, 0, 0, 0, 0, 115000, 0, 0, 0, 206000, 0, 68552, 0, 0, 0, 80000, 0, 0, 0, 0, 0, 0, 0, 0, 68552, 70127, 73211, 0, 0, 0, 0, 0, 90000, 0, 0, 0]
[32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 19

In [21]:
df = pd.read_csv('wnba-player-stats.csv')
df["Salaries"] = salaries_sorted
df["GS"] = games_started
df.to_csv('wnba-player-stats.csv')